# Amplicon silage time-trial

In [ ]:
#bash funtion and main path
def bash(string,name,queue):
  script = "%s.sh"%name
  with open(script, "w") as text_file:
    text_file.write(string)
  !chmod +x $script
  !qsub -q $queue $script
  !rm $script

ws = '/beegfs/work/workspace/ws/user-silage_timetrial-0/16srna/'
!rm -r raw_data/.ipynb_checkpoints

# Demultiplex with sabre

In [ ]:
#this was done by Timur
import pandas as pd

bc = pd.read_csv('Johan_Anthonia_samples.tsv', sep='\t') 
bc.Index = bc.Index.str.replace(' ', '_')
bc = bc[['Sample ID', 'BC', 'Index']]

bc.to_csv('barcodes.tsv', sep='\t', index=False) # save metadata

In [ ]:
import pandas as pd

met = pd.read_csv('barcodes.tsv',sep='\t',index_col=0)
for ri in met.Index.unique():
    !mkdir -p Demux/$ri
    meta = met.loc[met.Index == ri].copy()
    print(ri)
    display(meta)
    meta['r1'] = f'Demux/{ri}/' + meta.index +'_'+ meta.BC +'_L001_R1_001.fastq'
    meta['r2'] = f'Demux/{ri}/' + meta.index +'_'+ meta.BC +'_L001_R2_001.fastq'
    meta.set_index('BC', inplace=True)
    meta.drop('Index', axis=1, inplace=True)
    meta.to_csv(f'Demux/{ri}/bc.txt', sep='\t', header=False)
    bc = f'Demux/{ri}/bc.txt'
    #start here
    ind = ri.split('_')[-1]
    f = f'Mux/InsPPhyt_r2_i{ind}/*1.fq.gz'
    r = f'Mux/InsPPhyt_r2_i{ind}/*2.fq.gz'
    u = f'Demux/{ri}/no_bc_match_{ind}_R1.fastq'
    w = f'Demux/{ri}/no_bc_match_{ind}_R2.fastq'
    !sabre pe -m 0 -f $f -r $r -b $bc -u $u -w $w
    #!rm $bc
    !pigz Demux/$ri/*.fastq #faster alternative to gzip!

In [ ]:
#unzip data
!unzip -q 

# Import data

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path %s \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path %s
'''
in_folfer = ws+'raw_data/'
out = ws+'demux-paired-end.qza'
bash(string%(in_folfer, out),'import_qiime','short')

# Remove adapters

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=10:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime cutadapt trim-paired \
    --i-demultiplexed-sequences %s \
    --o-trimmed-sequences %s \
    --p-cores 14 \
    --p-front-f CAAGRGTTHGATYMTGGCTCAG \
    --p-front-r TGCTGCCTCCCGTAGGAGT \
    --p-match-adapter-wildcards \
    --p-discard-untrimmed \
    --p-match-read-wildcards
'''
!mkdir no_adaptor
in_file = ws+'demux-paired-end.qza'
out_file = ws+'no_adaptor/demux-no_adaptor.qza'
bash(string%(in_file, out_file),'demux_sum_qiime','short')

In [ ]:
# Create a summary to visualize
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime demux summarize \
  --i-data %s \
  --o-visualization %s
'''
in_folfer = ws+'no_adaptor/demux-no_adaptor.qza'
out = ws+'no_adaptor/demux-no_adaptor.qzv'
bash(string%(in_folfer, out),'demux_sum_qiime','short')

# Quality control and feature table construction

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime dada2 denoise-paired \
    --i-demultiplexed-seqs %s \
    --p-trunc-len-f 180 \
    --p-trunc-len-r 180 \
    --p-n-threads 14 \
    --o-table %s \
    --o-representative-sequences %s \
    --o-denoising-stats %s
'''

!mkdir dada2
in_file = ws+'no_adaptor/demux-no_adaptor.qza'
representative = ws+'dada2/rep-seqs-dada2.qza'
table = ws+'dada2/table-dada2.qza'
stats = ws+'dada2/stats-dada2.qza'
bash(string%(in_file, table, representative, stats),'dada2','short')

In [ ]:
# Create a summary to visualize
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime feature-table tabulate-seqs \
    --i-data %s \
    --o-visualization %s

qiime metadata tabulate \
    --m-input-file %s \
    --o-visualization %s

qiime feature-table summarize \
    --i-table %s \
    --o-visualization %s
'''

seqs = ws+'dada2/rep-seqs-dada2.qza'
out_seqs = ws+'dada2/rep-seqs-dada2.qzv'
stats = ws+'dada2/stats-dada2.qza' 
out_stats = ws+'dada2/stats-dada2.qzv'
table = ws+'dada2/table-dada2.qza'
out_table = ws+'dada2/table-dada2.qzv'
bash(string%(seqs, out_seqs, stats, out_stats, table,out_table),'dada2_summa','short')

# filter rare ASVs

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime feature-table filter-features \
    --i-table %s \
    --p-min-frequency 50 \
    --p-min-samples 2 \
    --o-filtered-table %s

qiime feature-table filter-seqs \
    --i-data %s \
    --i-table %s \
    --o-filtered-data %s
'''

!mkdir filtered_50_data
in_table = ws+'dada2/table-dada2.qza'
out_table = ws+'filtered_50_data/table-50_dada2.qza'
in_seq = ws+'dada2/rep-seqs-dada2.qza'
in_filtered_table = ws+'filtered_50_data/table-50_dada2.qza'
out_filtered_table =ws+'filtered_50_data/rep-seqs-50_dada2.qza'
bash(string%(in_table, out_table,in_seq, in_filtered_table, out_filtered_table),'filter_qiime','short')


# make a tree

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences %s \
  --o-alignment %s \
  --o-masked-alignment %s \
  --o-tree %s \
  --o-rooted-tree %s
'''
!mkdir tree

seqs = ws+'filtered_50_data/rep-seqs-50_dada2.qza'
aligment_out = ws+'tree/aligned-rep-seqs.qza'
masked_aligned = ws+'tree/masked-aligned-rep-seqs.qza'
unrooted_tree = ws+'tree/unrooted-tree.qza'
rooted_tree = ws+'tree/rooted-tree.qza'
bash(string%(seqs, aligment_out, masked_aligned, unrooted_tree, rooted_tree),'dada2_summa','short')

# core metrics

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny %s \
  --i-table %s \
  --p-sampling-depth 26000 \
  --m-metadata-file %s \
  --output-dir %s
'''

tree = ws+'tree/rooted-tree.qza'
table = ws+'filtered_50_data/table-50_dada2.qza'
metadata = ws+'amplicon_metadata.txt'
output = ws+'core_metrics'
bash(string%(tree, table, metadata, output),'core_qiime','short')

# Alpha rarefaction

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime diversity alpha-rarefaction \
  --i-table %s \
  --i-phylogeny %s \
  --p-max-depth 26000 \
  --m-metadata-file %s \
  --o-visualization %s
'''

table = ws+'filtered_50_data/table-50_dada2.qza'
tree = ws+'tree/rooted-tree.qza'
metadata = ws+'amplicon_metadata.txt'
output = ws+'alpha-rarefaction.qzv'
bash(string%(table, tree, metadata, output),'alpha_qiime','short')

# Classifier

In [ ]:
#donwload database
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime rescript get-silva-data \
  --p-version '138.1' \
  --p-target 'SSURef_NR99' \
  --p-include-species-labels \
  --o-silva-sequences %s \
  --o-silva-taxonomy %s
'''
!mkdir classifier
out_seq = ws+'classifier/silva-138_1-ssu-nr99-seqs.qza'
out_tax = ws+'classifier/silva-138_1-ssu-nr99-tax.qza'
bash(string%(out_seq, out_tax),'classiifer_qiime','short')

In [ ]:
# clean seqs
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime rescript cull-seqs \
  --i-sequences %s \
  --o-clean-sequences %s
'''

in_seq = ws+'classifier/silva-138_1-ssu-nr99-seqs.qza'
out_clean = ws+'classifier/silva-138_1-ssu-nr99-seqs-cleaned.qza'
bash(string%(in_seq, out_clean),'classiifer_clean','short')

In [ ]:
# filter seqs by size
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime rescript filter-seqs-length-by-taxon \
  --i-sequences %s \
  --i-taxonomy %s \
  --p-labels Archaea Bacteria Eukaryota \
  --p-min-lens 900 1200 1400 \
  --o-filtered-seqs %s \
  --o-discarded-seqs %s
'''

in_seq = ws+'classifier/silva-138_1-ssu-nr99-seqs-cleaned.qza'
taxonomy = ws+'classifier/silva-138_1-ssu-nr99-tax.qza'
out_filtered = ws+'classifier/silva-138_1-ssu-nr99-seqs-filt.qza'
out_dicarded = ws+'classifier/silva-138_1-ssu-nr99-seqs-discard.qza'
bash(string%(in_seq, taxonomy, out_filtered, out_dicarded),'classiifer_filter','short')

In [ ]:
# dereplicate seqs
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime rescript dereplicate \
  --i-sequences %s \
  --i-taxa %s \
  --p-rank-handles 'silva' \
  --p-mode 'uniq' \
  --o-dereplicated-sequences %s \
  --o-dereplicated-taxa %s
'''
in_filtered = ws+'classifier/silva-138_1-ssu-nr99-seqs-filt.qza'
taxonomy = ws+'classifier/silva-138_1-ssu-nr99-tax.qza'
out_drep = ws+'classifier/silva-138_1-ssu-nr99-seqs-derep-uniq.qza'
drep_tax = ws+'classifier/silva-138_1-ssu-nr99-tax-derep-uniq.qza'
bash(string%(in_filtered, taxonomy, out_drep, drep_tax),'classiifer_drep','short')

In [ ]:
# select V1-V2 region from seqs
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime feature-classifier extract-reads \
  --i-sequences %s \
  --p-f-primer CAAGRGTTHGATYMTGGCTCAG \
  --p-r-primer TGCTGCCTCCCGTAGGAGT \
  --p-n-jobs 14 \
  --p-read-orientation 'forward' \
  --o-reads %s
'''

in_seqs = ws+'classifier/silva-138_1-ssu-nr99-seqs-derep-uniq.qza'
out_v1v2 = ws+'classifier/silva-138_1-ssu-nr99-seqs-V1-V2.qza'
bash(string%(in_seqs, out_v1v2),'classiifer_v1v2','short')

In [ ]:
# dereplicate V1-V2 region seqs
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime rescript dereplicate \
  --i-sequences %s  \
  --i-taxa %s \
  --p-rank-handles 'silva' \
  --p-mode 'uniq' \
  --o-dereplicated-sequences %s \
  --o-dereplicated-taxa %s  
'''

in_seqs = ws+'classifier/silva-138_1-ssu-nr99-seqs-V1-V2.qza'
drep_tax = ws+'classifier/silva-138_1-ssu-nr99-tax-derep-uniq.qza'
out_drep = ws+'classifier/silva-138_1-ssu-nr99-seqs-V1-V2-uniq.qza'
out_drep_tax = ws+'classifier/silva-138_1-ssu-nr99-tax-V1-V2-derep-uniq.qza'
bash(string%(in_seqs, drep_tax, out_drep, out_drep_tax),'classiifer_v1v2_drep','short')

In [ ]:
# Fit the classifier
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads %s \
  --i-reference-taxonomy %s \
  --o-classifier %s
'''
in_seqs = ws+'classifier/silva-138_1-ssu-nr99-seqs-V1-V2-uniq.qza'
in_tax = ws+'classifier/silva-138_1-ssu-nr99-tax-V1-V2-derep-uniq.qza'
out_seqs = ws+'classifier/V1-V2-ssu-nr99-classifier.qza'
bash(string%(in_seqs, in_tax, out_seqs),'classiifer_fitting','short')

# Classify sequences

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime feature-classifier classify-sklearn \
  --i-classifier %s \
  --i-reads %s \
  --o-classification %s
  
qiime metadata tabulate \
  --m-input-file %s \
  --o-visualization %s
'''

!mkdir taxonomy
classifier = ws+'classifier/V1-V2-ssu-nr99-classifier.qza'
rep_seqs = ws+'filtered_50_data/rep-seqs-50_dada2.qza'
out_tax = ws+'taxonomy/taxonomy.qza'
out_tax_visual= ws+'taxonomy/taxonomy.qzv'
bash(string%(classifier, rep_seqs, out_tax, out_tax, out_tax_visual),'tax_qiime','short')

# Barplots qiime

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime taxa barplot \
  --i-table %s \
  --i-taxonomy %s \
  --m-metadata-file %s \
  --o-visualization %s
'''

in_table = ws+'core_metrics/rarefied_table.qza'
taxonomy = ws+'taxonomy/taxonomy.qza'
metadata = ws+'amplicon_metadata.txt'
barplots = ws+'taxa-bar-plots.qzv'
bash(string%(in_table, taxonomy, metadata, barplots),'barplot_qiime','short')

# Export files

In [ ]:
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=00:19:00
#PBS -l mem=1gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime tools export \
  --input-path %s \
  --output-path %s
'''
!mkdir exports
in_file = ws+'core_metrics/rarefied_table.qza'
out_file = ws+'exports/rarefied_table'
bash(string%(in_file, out_file),'export_qiime','tiny')

In [ ]:
# convert the BIOM table to a tsv 
biom convert -i feature-table.biom -o feature-table.tsv --to-tsv

# Adonis test

In [ ]:
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=00:19:00
#PBS -l mem=1gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime diversity beta-group-significance \
  --i-distance-matrix %s \
  --m-metadata-file %s \
  --m-metadata-column day \
  --p-pairwise \
  --output-dir %s
'''
!mkdir statistical_test
matrix = ws+'core_metrics/unweighted_unifrac_distance_matrix.qza'
metadata = ws+'amplicon_metadata.txt'
out = ws+'statistical_test/adonis'
bash(string%(matrix, metadata, out),'adonis_qiime','tiny')

# Alpha-group-significance

In [ ]:
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=00:19:00
#PBS -l mem=1gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/user-conda-0/conda/etc/profile.d/conda.sh
conda activate qiime2-2022.8

qiime diversity alpha-group-significance \
  --i-alpha-diversity %s \
  --m-metadata-file %s \
  --o-visualization %s 

qiime diversity alpha-group-significance \
  --i-alpha-diversity %s \
  --m-metadata-file %s \
  --o-visualization %s
'''

in_features = ws+'core_metrics/observed_features_vector.qza'
metadata = ws+'amplicon_metadata.txt'
out_features = ws+'statistical_test/features-group-significance.qzv'

in_shannon = ws+'core_metrics/shannon_vector.qza'
out_shannon = ws+'statistical_test/shannons-group-significance.qzv'
bash(string%(in_features, metadata, out_features,in_shannon, metadata,out_shannon),'alpha_qiime','tiny')